<a href="https://colab.research.google.com/github/SadeghMahmoudAbadi/Open-Source-LLM-on-Colab/blob/main/Company_Brochure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ollama

In [2]:
from google.colab import userdata
from ollama import Client
import json
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display

In [3]:
# Standard headers to fetch a website
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [4]:
def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url;
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)

In [5]:
def fetch_website_links(url):
    """
    Return the links on the webiste at the given url
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    links = [link.get("href") for link in soup.find_all("a")]
    return [link for link in links if link]

In [6]:
links = fetch_website_links("https://huggingface.co/")
links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/moonshotai/Kimi-K2-Thinking',
 '/dx8152/Qwen-Edit-2509-Multiple-angles',
 '/MiniMaxAI/MiniMax-M2',
 '/maya-research/maya1',
 '/deepseek-ai/DeepSeek-OCR',
 '/models',
 '/spaces/HuggingFaceTB/smol-training-playbook',
 '/spaces/tori29umai/Qwen-Image-2509-MultipleAngles',
 '/spaces/linoyts/Qwen-Image-Edit-Angles',
 '/spaces/dream2589632147/Dream-wan2-2-faster-Pro',
 '/spaces/Wan-AI/Wan2.2-Animate',
 '/spaces',
 '/datasets/nvidia/PhysicalAI-Autonomous-Vehicles',
 '/datasets/HuggingFaceFW/finewiki',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/neulab/agent-data-collection',
 '/datasets/Open-Bee/Honey-Data-15M',
 '/datasets',
 '/join',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/inference/models',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/allenai',
 '/facebook',
 '/ama

In [7]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
def get_links_user_prompt(url):
    user_prompt = f"""
    Here is the list of links on the website {url} -
    Please decide which of these are relevant web links for a brochure about the company,
    respond with the full https URL in JSON format.
    Do not include Terms of Service, Privacy, email links.

    Links (some might be relative links):

    """
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [9]:
print(get_links_user_prompt("https://huggingface.co/"))


    Here is the list of links on the website https://huggingface.co/ -
    Please decide which of these are relevant web links for a brochure about the company, 
    respond with the full https URL in JSON format.
    Do not include Terms of Service, Privacy, email links.

    Links (some might be relative links):

    /
/models
/datasets
/spaces
/docs
/enterprise
/pricing
/login
/join
/spaces
/models
/moonshotai/Kimi-K2-Thinking
/dx8152/Qwen-Edit-2509-Multiple-angles
/MiniMaxAI/MiniMax-M2
/maya-research/maya1
/deepseek-ai/DeepSeek-OCR
/models
/spaces/HuggingFaceTB/smol-training-playbook
/spaces/tori29umai/Qwen-Image-2509-MultipleAngles
/spaces/linoyts/Qwen-Image-Edit-Angles
/spaces/dream2589632147/Dream-wan2-2-faster-Pro
/spaces/Wan-AI/Wan2.2-Animate
/spaces
/datasets/nvidia/PhysicalAI-Autonomous-Vehicles
/datasets/HuggingFaceFW/finewiki
/datasets/fka/awesome-chatgpt-prompts
/datasets/neulab/agent-data-collection
/datasets/Open-Bee/Honey-Data-15M
/datasets
/join
/enterprise
/enterpri

In [10]:
MODEL = "gpt-oss:120b"
ollama = Client(
    host="https://ollama.com",
    headers={'Authorization': 'Bearer ' + userdata.get("OLLAMA_API_KEY")}
)

In [11]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = ollama.chat(
        MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        format={"type": "json_object"}
    )
    result = response['message']['content']
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [12]:
select_relevant_links("https://huggingface.co/")

Selecting relevant links for https://huggingface.co/ by calling gpt-oss:120b
Found 10 relevant links


{'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'learn page', 'url': 'https://huggingface.co/learn'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'github', 'url': 'https://github.com/huggingface'},
  {'type': 'twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'linkedin',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

In [13]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [14]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-oss:120b
Found 7 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
moonshotai/Kimi-K2-Thinking
Updated
about 5 hours ago
•
12.5k
•
626
dx8152/Qwen-Edit-2509-Multiple-angles
Updated
2 days ago
•
16.7k
•
364
MiniMaxAI/MiniMax-M2
Updated
1 day ago
•
854k
•
1.18k
maya-research/maya1
Updated
1 day ago
•
4.58k
•
288
deepseek-ai/DeepSeek-OCR
Updated
4 days ago
•
2.89M
•
2.54k
Browse 1M+ models
Spaces
Running
on
CPU Upgrade
1.71k
1.71k
The Smol Training Playbook: The Secrets to Building World-Class LLMs
📝
Running
on
Zero
241
241
Qwen-Image-2509-MultipleAngles
👀
Qwen-Image-2509-MultipleAngles
Running
on
Zer

In [15]:
brochure_system_prompt = """
You are a snarky assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
    You are looking at a company called: {company_name}
    Here are the contents of its landing page and other relevant pages;
    use this information to build a short brochure of the company in markdown without code blocks.\n\n
    """
    user_prompt += fetch_page_and_all_relevant_links(url)
    # user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-oss:120b
Found 11 relevant links


'\n    You are looking at a company called: HuggingFace\n    Here are the contents of its landing page and other relevant pages;\n    use this information to build a short brochure of the company in markdown without code blocks.\n\n\n    ## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmoonshotai/Kimi-K2-Thinking\nUpdated\nabout 5 hours ago\n•\n12.5k\n•\n626\ndx8152/Qwen-Edit-2509-Multiple-angles\nUpdated\n2 days ago\n•\n16.7k\n•\n364\nMiniMaxAI/MiniMax-M2\nUpdated\n1 day ago\n•\n854k\n•\n1.18k\nmaya-research/maya1\nUpdated\n1 day ago\n•\n4.58k\n•\n288\ndeepseek-ai/DeepSeek-OCR\nUpdated\n4 days ago\n•\n2.89M\n•\n2.54k\nBrowse 1M+ models\nSpaces\nRunning\no

In [18]:
def create_brochure(company_name, url):
    response = ollama.chat(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response['message']['content']
    display(Markdown(result))

In [20]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-oss:120b
Found 9 relevant links


# 🤗 Hugging Face – The AI Community That Actually *Hugs* Back  

---

## 🌟 Who Are We?  

- **Mission:** “The AI community building the future.” In other words, we’re the **friendliest** place on the internet where models, datasets, and apps mingle like a techy cocktail party.  
- **Founded:** 2016 (still young enough to be *hip*, old enough to have *real* AI cred).  
- **Headquarters:** Paris & NYC – because we like croissants **and** pretzels.  
- **Team Size:** 51‑200 brilliant, caffeinated humans (plus an army of open‑source bots).  

---

## 🏢 The Playground (aka “The Hub”)  

| What you’ll find | Why it’s cool (and slightly addictive) |
|------------------|----------------------------------------|
| **1 M+ models** | From text‑only chatterboxes to video‑generating magicians – browse, fork, or just stare in awe. |
| **250 k+ datasets** | Anything from autonomous‑vehicle sensor logs to “awesome‑ChatGPT‑prompts” (yes, we love prompts). |
| **400 k+ Spaces** | One‑click apps running on CPU, GPU, or “Zero” (the mystical realm where all compute is free‑ish). |
| **9 k+ open‑source libs** | Transformers, Diffusers, PEFT, Accelerate… basically every tool you’ll need to sound smart at a conference. |

> **Pro‑Tip:** “ZeroGPU” is a thing. We call it *Zero‑to‑hero* for your model’s performance.

---

## 👯‍♀️ Culture (The “Hug” in Hugging Face)

- **Open‑source first:** We treat pull‑requests like love letters.  
- **Community‑driven:** 50 000+ orgs (Meta, Google, Amazon, Intel, Microsoft…) already share their toys with us.  
- **Snack‑friendly:** We sponsor hackathons with $15k cash prizes + $1.3 M in API credits – because nothing fuels code like free money.  
- **Ethics‑obsessed:** “Toward Community‑Governed Safety” isn’t a bedtime story; it’s a living doc.  

> *“We’re not just building AI; we’re building trust, one repo at a time.”* — Rumor has it these are the actual words of a senior engineer on Slack.  

---

## 🏢 Who’s Already Hugging Us?

| Customer | Models Hosted | Followers |
|----------|---------------|-----------|
| **Meta (AI at Meta)** | 2 230 | 8.6 k |
| **Google** | 1 050 | 34.8 k |
| **Microsoft** | 426 | 16.3 k |
| **Amazon** | 21 | 3.5 k |
| **NVIDIA** | 577 | 42.1 k |
| **Grammarly** | 11 | 187 |
| **Shopify** | 580 (enterprise) | — |
| **AI2 (Allen Institute)** | 805 | 4.3 k |

*…and *more than 45 000 other daring explorers* who have already set up shop on our platform.*

---

## 💰 Pricing (Because even hugging costs a little)

| Plan | Price | What you get |
|------|-------|--------------|
| **Free** | $0 | Unlimited public models, datasets, Spaces on CPU. |
| **Team** | **$20 / user / month** | SSO, audit logs, private dataset viewer, priority support. |
| **Enterprise** | Custom | All the above **plus** regional data controls, private storage (1 TB per user), ZeroGPU boost, dedicated success manager. |
| **Compute** | Starting **$0.60 / hour** for GPU | Spin up a GPU in a click; perfect for that one‑off “train‑my‑model‑in‑the‑cloud” binge. |

> **Bottom line:** If you can afford a latte, you can afford a Hugging Face Space.

---

## 🚀 Careers (Join the Hug Party)

- **Open positions:** Software Engineers, ML Researchers, Product Managers, Community Educators, **and** a *Full‑Stack Engineer who can make our UI look as friendly as a golden retriever*.  
- **Why work here?**  
  - Work on the *very* same libraries (Transformers, Diffusers, …) that power the world’s biggest AI products.  
  - Get to brag about “I contribute to the most‑starred 🤗 repo on GitHub (152 k⭐)”.  
  - Participate in a culture where **hackathons** are a lifestyle, **open‑source** is a religion, and **snacks** are a right.  
- **Location flexibility:** Remote‑first, with occasional meet‑ups in Paris, NYC, or wherever the next AI conference is held.  

> *“I joined Hugging Face for the code. I stay for the memes.”* — Anonymous (probably a senior dev).

---

## 📚 Learning (Because you can’t hug what you don’t know)

- **LLM Course, Diffusion Course, RL Course, 3D ML, Audio…** everything you need to become the next AI wizard.  
- **Open‑Source AI Cookbook** – a treasure chest of notebooks curated by the community.  
- **Community‑run webinars** (the “MCP Birthday Party” is basically a conference + karaoke).  

---

## 🎉 The Bottom Line

If you want a **platform** where you can **collaborate**, **experiment**, and **show off** your AI wizardry *without a mountain of legal paperwork*, Hugging Face is the friendly neighbourhood AI hub you’ve been dreaming about.  

**Ready to get your hands (and code) on a hugging face?**  

➡️ **Sign up** → **Explore 1 M+ models** → **Deploy a Space** → **Collect bragging rights**  

*Because in a world of cold, hard silicon, we’re the only ones who actually give you a hug.* 🤗

In [21]:
def stream_brochure(company_name, url):
    stream = ollama.chat(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk['message']['content'] or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-oss:120b
Found 10 relevant links


# 🤗 **Hugging Face – The Friendly AI Playground**  

*“We’re the AI community building the future, one 🤗 emoji at a time.”*  

---  

## 🌍 Who (or what) is Hugging Face?  

- **A one‑stop‑shop** for **models, datasets, Spaces (apps)**, and a buzzing community that treats open‑source like it’s a potluck dinner – everyone brings something tasty.  
- **Founded 2016**, now a *51‑200* employee boutique perched in **Paris & NYC**, with a worldwide fan‑club that includes the likes of **Google, Meta, Amazon, Microsoft, Intel, NVIDIA, & the occasional non‑profit**.  
- **Mission:** Democratize AI so even your grandma can fine‑tune a language model (or at least **pretend** she can).  

---  

## 🧩 The Hugging Face Stack – “The Swiss‑Army Knife of AI”

| Toolbox | What It Does | Fun Fact |
|---------|--------------|----------|
| **🤗 Transformers** | State‑of‑the‑art model zoo (text, vision, audio, multimodal) | >150 k stars on GitHub – more than the average Hollywood star’s fan club |
| **🤗 Diffusers** | Diffusion models for images, video, audio | Turns “pixel art” into “pixel *art*” in seconds |
| **🤗 Datasets** | 250 k+ ready‑to‑use datasets (including the legendary `awesome‑chatgpt‑prompts`) | You’ll find more data here than on your ex’s Dropbox |
| **🤗 Tokenizers** | Lightning‑fast tokenizers for research & production | Faster than a coffee‑drunk intern on a deadline |
| **🤗 PEFT** | Parameter‑efficient fine‑tuning (big models, tiny budgets) | Because who wants to spend a bazillion dollars on GPUs? |
| **🤗 Accelerate / Optimum** | One‑click multi‑GPU/TPU scaling, mixed‑precision, FP8 | “Zero‑GPU” vs “Zero‑effort” – you pick the magic word |
| **🤗 Spaces** | Host & share live AI demos (no Docker required) | Deploy on a CPU, upgrade to GPU with a click, and watch the watchers applaud |

---  

## 🚀 “Trending This Week” – The Cool Kids on the Block  

- **Kimi‑K2‑Thinking** – 12.5 k likes, 626 forks.  
- **DeepSeek‑OCR** – 2.89 M downloads, because everyone still needs to read PDFs.  
- **MiniMax‑M2** – 854 k likes, proof that “mini” can still be massive.  

*(If you don’t see your favorite model, it’s probably sleeping in the “unlimited public repo” lounge.)*  

---  

## 🤝 Community & Culture – Where Nerds Meet Nice  

- **Open‑source first.** All core libraries live on GitHub under permissive licenses; PRs are welcomed faster than a coffee refill at a hackathon.  
- **Hackathons & MCP parties.** Over **$15k** in cash prizes, **$1.3 M** in API credits, and a virtual cake‑throwing ceremony.  
- **Slack‑ish Discord** where newbies ask “How do I pay my inference usage?” and veterans answer with a meme and a solution.  
- **Cheeky Blog & Courses** (LLM, Robotics, Diffusion, 3‑D…) that make you smarter *and* keep you humble enough to admit you still need Stack Overflow.  

> *“If you can’t find a solution, just ask the community. If they can’t, they’ll write a new library for you.”*  

---  

## 🏢 Enterprise Clients – Who’s Paying the Bills?  

| Customer | #Models Hosted | Followers |
|----------|----------------|-----------|
| **Meta (AI at Meta)** | 2.23 k | 8.64 k |
| **Google** | 1.05 k | 34.8 k |
| **Microsoft** | 426 | 16.3 k |
| **Amazon** | 21 | 3.53 k |
| **Intel** | 250 | 3.19 k |
| **NVIDIA** | 577 | 42.1 k |
| **Grammarly** | 11 | 187 |
| **Airbnb** | 102 | — |
| …and 50 000+ other orgs (including non‑profits like **AI2** & **AI for Good**).  

*Enterprise plans start at **$20 / user / month** with SSO, audit logs, private datasets, and a dedicated support team that actually reads your tickets.*  

---  

## 💼 Careers – Want to Join the Friendly AI Circus?  

- **Roles:** Machine‑Learning Engineer, Research Scientist, Software Engineer, Product Manager, Community Manager, *and* the ever‑mysterious “Special Projects” (rumoured to involve cats and lasers).  
- **Perks:**  
  - Unlimited public repos → bragging rights on GitHub.  
  - Access to **ZeroGPU** (make your code run on “nothing” – it’s a thing).  
  - Quarterly hackathon cash prizes (because nothing says “we value you” like a sudden $10k).  
  - Remote‑first culture – work from Paris, NYC, or your couch.  
- **Hiring page:** *Currently jam‑packed with 555 k+ open jobs across the AI universe. Pick one, or pick a few, or just RSVP to the next virtual party.*  

---  

## 📈 Why Investors Should Care  

1. **Network Effect:** Over **45 k+ models** accessible via a single API → stickiness that makes churn as rare as a bug‑free release.  
2. **Revenue Engine:** Enterprise tier, paid compute (GPU @ $0.60 / hour), and inference provider billing.  
3. **Open‑Source Credibility:** The go‑to libraries for transformers and diffusers – everyone from startups to Fortune‑500s depends on them.  
4. **Growth Momentum:** *50 k+ organizations* already on board, plus a pipeline of new models added daily.  

*In short: Hugging Face is the **Spotify of AI models** – a massive catalog, social discovery, and a subscription model that keeps the lights on.*  

---  

## 📞 Get In Touch  

- **Website:** https://huggingface.co  
- **Docs & Playground:** Docs → “Explore all modalities: Text, Image, Video, Audio, 3‑D.”  
- **Social:**  
  - X (Twitter) – where the team drops memes and API credits.  
  - LinkedIn – 551 employees, counting.  
  - Discord – the place where “index -1” errors are solved before coffee.  

**Ready to hug a model?** Press **Sign Up**, spin up a Space, and join the most *friendly* AI community on the planet.  

---  

*Disclaimer: Hugging Face does not guarantee that you will actually become a world‑renowned AI wizard. It does, however, guarantee a steady stream of emojis, community love, and the occasional “I built a robot that can write poetry” brag.*